In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/contradictory-my-dear-watson/sample_submission.csv
/kaggle/input/contradictory-my-dear-watson/train.csv
/kaggle/input/contradictory-my-dear-watson/test.csv


In [2]:
# import other libraries
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset
import torch
from sklearn.metrics import accuracy_score, f1_score
import tensorflow as tf

2025-10-02 05:13:05.765470: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1759381986.020956      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1759381986.100609      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
# load tokenizer and model
model_name = "joeddav/xlm-roberta-large-xnli"
tokenizer = AutoTokenizer.from_pretrained(model_name)
device = "cuda" if torch.cuda.is_available() else "cpu"

model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=3)
model.config.hidden_dropout_prob = 0.2
model.to(torch.device(device))

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/734 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

Some weights of the model checkpoint at joeddav/xlm-roberta-large-xnli were not used when initializing XLMRobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


XLMRobertaForSequenceClassification(
  (roberta): XLMRobertaModel(
    (embeddings): XLMRobertaEmbeddings(
      (word_embeddings): Embedding(250002, 1024, padding_idx=1)
      (position_embeddings): Embedding(514, 1024, padding_idx=1)
      (token_type_embeddings): Embedding(1, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): XLMRobertaEncoder(
      (layer): ModuleList(
        (0-23): 24 x XLMRobertaLayer(
          (attention): XLMRobertaAttention(
            (self): XLMRobertaSdpaSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): XLMRobertaSelfOutput(
              (dense): Linear(in_features=1024, ou

In [4]:
# set up train, test dfs
train_df = pd.read_csv('/kaggle/input/contradictory-my-dear-watson/train.csv')
test_df = pd.read_csv('/kaggle/input/contradictory-my-dear-watson/test.csv')

In [5]:
# check columns
print(f'train_df columns: {train_df.columns}')
print(f'train_df shape: {train_df.shape}')
print(f'test_df columns: {test_df.columns}')
print(f'test_df shape: {test_df.shape}')

train_df columns: Index(['id', 'premise', 'hypothesis', 'lang_abv', 'language', 'label'], dtype='object')
train_df shape: (12120, 6)
test_df columns: Index(['id', 'premise', 'hypothesis', 'lang_abv', 'language'], dtype='object')
test_df shape: (5195, 5)


In [6]:
# model config is 0: contradiction, 1: neutral, 2: entailment
# reverse this for this competition

comp_to_hf = {0:2, 1:1, 2:0}
hf_to_comp = {v:k for k,v in comp_to_hf.items()}

In [7]:
# force label to integer
train_df['label'] = train_df['label'].astype(int)

# drop missing values
train_df = train_df.dropna(subset = ['label'])

# convert competition labels to HF numeric ids
train_df['label'] = train_df['label'].map(comp_to_hf)

In [8]:
# tokenize dataset
max_len = 128

# helper function to map tokenizer
def tokenize(df):
    return tokenizer(df['premise'], df['hypothesis'], truncation = True, padding = 'max_length', max_length = max_len)

train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

train_dataset = train_dataset.map(tokenize, batched = True)
test_dataset = test_dataset.map(tokenize, batched = True)

Map:   0%|          | 0/12120 [00:00<?, ? examples/s]

Map:   0%|          | 0/5195 [00:00<?, ? examples/s]

In [9]:
train_dataset.features

{'id': Value(dtype='string', id=None),
 'premise': Value(dtype='string', id=None),
 'hypothesis': Value(dtype='string', id=None),
 'lang_abv': Value(dtype='string', id=None),
 'language': Value(dtype='string', id=None),
 'label': Value(dtype='int64', id=None),
 'input_ids': Sequence(feature=Value(dtype='int32', id=None), length=-1, id=None),
 'attention_mask': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None)}

In [10]:
test_dataset.features

{'id': Value(dtype='string', id=None),
 'premise': Value(dtype='string', id=None),
 'hypothesis': Value(dtype='string', id=None),
 'lang_abv': Value(dtype='string', id=None),
 'language': Value(dtype='string', id=None),
 'input_ids': Sequence(feature=Value(dtype='int32', id=None), length=-1, id=None),
 'attention_mask': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None)}

In [11]:
# helper function to compute metrics
def compute_metrics(p):
    preds = np.argmax(p.predictions, axis=1)
    return {
        'accuracy': accuracy_score(p.label_ids, preds),
        'f1': f1_score(p.label_ids, preds, average='weighted')
    }

In [12]:
# training arguments setup for trainer
training_args = TrainingArguments(
    output_dir = '/kaggle/results',
    run_name="xlm-roberta-nli-run1",
    eval_strategy = 'epoch',
    save_strategy = 'epoch',
    learning_rate = 2e-5,
    per_device_train_batch_size = 8,
    per_device_eval_batch_size = 8,
    gradient_accumulation_steps = 2,
    num_train_epochs = 3,
    weight_decay = 0.01,
    fp16 = True,
    logging_steps = 100,
    logging_strategy = 'steps',
    report_to = 'none',
    save_total_limit = 1,
    load_best_model_at_end = True,
    metric_for_best_model = 'accuracy',
)

In [13]:
trainer = Trainer(
    model = model,
    args = training_args,
    train_dataset = train_dataset.shuffle(seed=38).select(range(int(0.9*len(train_dataset)))),
    eval_dataset = train_dataset.shuffle(seed=38).select(range(int(0.9*len(train_dataset)), len(train_dataset))),
    processing_class = tokenizer,
    compute_metrics = compute_metrics
)

In [14]:
trainer.train()

/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.260500,0.207228,0.926568,0.926962
2,0.140400,0.245020,0.927393,0.927466
3,0.151400,0.727619,0.925743,0.925914


/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


TrainOutput(global_step=1023, training_loss=0.17672157870243255, metrics={'train_runtime': 2313.1574, 'train_samples_per_second': 14.147, 'train_steps_per_second': 0.442, 'total_flos': 7624156243719168.0, 'train_loss': 0.17672157870243255, 'epoch': 3.0})

In [15]:
# prepare test dataset for prediction
test_dataset.set_format(type="torch")

preds_output = trainer.predict(test_dataset)
hf_preds = np.argmax(preds_output.predictions, axis=-1)

In [16]:
# map HF predictions to competition label ids
hf_to_comp = {2:0, 1:1, 0:2}
comp_preds = [hf_to_comp[int(p)] for p in hf_preds]

In [17]:
# create submission csv
submission = pd.DataFrame({"id": test_dataset["id"], "prediction": comp_preds})
submission.to_csv("submission.csv", index=False)